<a href="https://colab.research.google.com/github/Kingzel/pyMD/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import csv
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [3]:
from setup import gen_multichoice_features , gen_binary_features , gen_singlechoice_features

Read Evidences/Symptoms

In [4]:
evids = pd.read_json("data\\release_evidences.json")

From the evidences, prepare to explode the predictors for binary categorical, multi choice categroical and single choice categorical evidences accoridngly to be interger encoded (one hot encoding + interger ordinal encoding where neccessary)

In [5]:
m_collection = gen_multichoice_features(evids)
c_collection =gen_singlechoice_features(evids)
b_collection = gen_binary_features(evids)

Manually reincluding SEX and AGE as predictors.

In [6]:
all_collections = b_collection+m_collection+c_collection[0]
all_collections.extend(['AGE','SEX'])
print(len(all_collections))



914


Import training dataset batches and slice into (predictors and target)

In [7]:
from setup import parse_data
parsed = parse_data(["output\\output_1.csv","output\\output_2.csv","output\\output_3.csv","output\\output_4.csv","output\\output_5.csv"])
train_patients,y = pd.concat(parsed[0::2], ignore_index=True),pd.concat(parsed[1::2],ignore_index=True)

Creating a dummy feature space with predictors as accumulated in `all_collections` and populating with `len(train_patients)` number of rows filled with 0s. 

In [8]:
from setup import gen_empty
new_df = gen_empty(all_collections,len(train_patients)) 

#Verifying shape of the dummy feature space and the values of the first few rows and columns.
print(new_df.shape,"\n")
print(new_df.iloc[0:2,:4])

(150000, 914) 

   E_91  E_53  E_159  E_129
0     0     0      0      0
1     0     0      0      0


Actually populating the feature space with patient data (symptoms, sex and age are loaded in structural accordance.)

In [9]:
from setup import populate
new_df = populate(new_df,train_patients)

In [10]:
#Verifying shape of the dummy feature space and the values of the first few rows and columns.
print(new_df.shape)
print(new_df.iloc[0:2,:4])

(150000, 914)
   E_91  E_53  E_159  E_129
0     1     1      0      0
1     1     1      0      1


Splitting the dataset into training and testing sets.

In [11]:

X_train, X_test, y_train, y_test = train_test_split(new_df, y, test_size = 0.35)

Fitting a Random Forest Classifier on the training set (10 trees)

In [199]:
clf = RandomForestClassifier(n_estimators=25, warm_start=True,max_depth=15)
clf.fit(X_train, y_train)
print(len(X_train))


97500


Predicting and evlauating classification accuracy of the RF model.

In [200]:
y_pred  = clf.predict(X_test)
correct = 0
total =0
for predicted, actual in zip(y_pred,y_test):
  if predicted == actual:
    correct+=1
  total+=1
print(correct,'out of',total,'\n',correct/total,'accuracy')

51653 out of 52500 
 0.9838666666666667 accuracy


Saving the trained model using pickle binary dumps.

In [201]:
import pickle as p

with open('trained_model.bin', 'wb') as f:
    p.dump(clf, f)
    p.dump(all_collections,f)


<h1>TESTING AND DEPLOYMENT</H1>

In [202]:
import pickle as p
with open('trained_model.bin', 'rb') as f:
    rf =  p.load(f)
    cols  = p.load(f)
print(len(cols))

914


In [43]:
from setup import parse_data , populate, gen_empty
parsed = parse_data(["data\\release_test_patients\\release_test_patients.csv"])
test_patients = parsed[0]
y=parsed[1]
new_df = gen_empty(cols,len(test_patients))


In [46]:
new_df = populate(new_df,test_patients)

In [203]:
y_pred  = rf.predict(new_df)
correct = 0
total =0
for predicted, actual in zip(y_pred,y):
  if predicted == actual:
    correct+=1
  total+=1
print(correct,'out of',total,correct/total,'accuracy')
# print(new_df.head())

132253 out of 134529 0.9830817147232195 accuracy


In [51]:

import visualize
import importlib
importlib.reload(visualize)
num_of_trees = len(rf.estimators_)
for i in range(2):
    visualize.tvisual(10,cols,rf.estimators_[i])

In [215]:
import operator
dummy = gen_empty(cols,1)
# print(dummy['E_104'])
# dummy['E_178'] = 1
# # dummy['E_41'] = 1
# # dummy['E_73'] =1
# # dummy['E_201'] =1
# dummy['E_66'] =1
# # dummy['E_57'] =1
# dummy['E_220'] =1
# dummy['E_39'] =1
# dummy['E_194'] =1
# dummy['E_124']=1
# # dummy['E_79']=1
# dummy['E_53']=1
# dummy['E_222']=1
# dummy['E_125']=1
dummy['E_116']=1
dummy['E_204_@_V_2']=1
dummy['E_56']=4

# "E_124": {},
#       "E_116": {},
#       "E_79": {},
#       "E_222": {},
#       "E_125": {},
#       "E_204": {}


dummy['AGE'] = 60

# dummy['E_97'] =1
# dummy['SEX']=1

# dummy['E_204_@_V_1'] = 1

# dummy['E_57_@_V_95'] = 1
# print(dummy['E_104'])



y_probabs = rf.predict_proba(dummy)
y_predict = rf.predict(dummy)
zipped = zip(rf.classes_,y_probabs.tolist()[0])
prob_to_label ={}
for disease,probab in zipped:
    prob_to_label[disease] = probab
prob_to_label = dict( sorted(prob_to_label.items(), key=operator.itemgetter(1), reverse=True))

print(prob_to_label,y_predict,sep = '\n'*2)



{'Bronchospasm / acute asthma exacerbation': 0.077532687863089, 'Larygospasm': 0.05391012191119759, 'Bronchiectasis': 0.05014511270253456, 'Allergic sinusitis': 0.03944151612202885, 'Acute COPD exacerbation / infection': 0.0371035886991858, 'Tuberculosis': 0.0367841575019686, 'Bronchitis': 0.030181484400746014, 'URTI': 0.029922566381589383, 'Pericarditis': 0.029663787482013246, 'Myasthenia gravis': 0.02953954349840544, 'Unstable angina': 0.029252945562480034, 'Possible NSTEMI / STEMI': 0.029175026012739514, 'Atrial fibrillation': 0.028613645261877006, 'Guillain-Barré syndrome': 0.027747809057128477, 'Stable angina': 0.027152480247363272, 'Acute dystonic reactions': 0.026307676193852592, 'Acute otitis media': 0.02628779089003069, 'Boerhaave': 0.025084367630557516, 'GERD': 0.022944889280576204, 'Whooping cough': 0.02287769772238175, 'PSVT': 0.022521610088828727, 'Cluster headache': 0.02144504272922758, 'Myocarditis': 0.01989674481779404, 'Anemia': 0.019819167131795136, 'Pulmonary neoplas

DONE